In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
dbutils.widgets.text("env","")
dbutils.widgets.text("client_id","")
dbutils.widgets.text("tax_year","")
env=dbutils.widgets.get("env")
tax_year=dbutils.widgets.get("tax_year")
client_id=dbutils.widgets.get("client_id")

In [0]:
import sys
sys.path.append("/Workspace/Users/mayur10594@gmail.com/ETL_project")

In [0]:
from config.utils import get_logger
logger = get_logger("bronze_notebook")

In [0]:
import json, yaml, os
from config.utils import load_config,create_tables

In [0]:
path="/Workspace/Users/mayur10594@gmail.com/ETL_project/config/etl_main.yaml"
env_config,tables_config=load_config(path,env)

In [0]:
in_path=env_config['incoming_path']
bronze_path=env_config['bronze_path']
logger.info(f"Loading data from {in_path}")
logger.info(f"bronze_path is set to {bronze_path}")
logger.info(f"env is set to {env}")
logger.info(f"tax_year is set to {tax_year}")
logger.info(f"client_id is set to {client_id}")

In [0]:
dbutils.fs.ls(in_path)

In [0]:
df_trans=spark.read.csv(f"{in_path}/transactions_data.csv",header=True)
logger.info(f"df_trans has {df_trans.count()} rows")
df_users=spark.read.csv(f"{in_path}/users_data.csv",header=True)
logger.info(f"df_users has {df_users.count()} rows")
df_cards=spark.read.csv(f"{in_path}/cards_data.csv",header=True)
logger.info(f"df_cards has {df_cards.count()} rows")

In [0]:
from datetime import date
date_str = date.today().strftime("%Y-%m-%d")

In [0]:
df_trans.write.mode("overwrite").format("parquet").save(f"{bronze_path}/{tax_year}/{client_id}/{date_str}/transactions_data")
logger.info(f"df_trans saved to {bronze_path}/{tax_year}/{client_id}/{date_str}/transactions_data")
df_users.write.mode("overwrite").format("parquet").save(f"{bronze_path}/{tax_year}/{client_id}/{date_str}/users_data")
logger.info(f"df_users saved to {bronze_path}/{tax_year}/{client_id}/{date_str}/users_data")
df_cards.write.mode("overwrite").format("parquet").save(f"{bronze_path}/{tax_year}/{client_id}/{date_str}/cards_data")
logger.info(f"df_cards saved to {bronze_path}/{tax_year}/{client_id}/{date_str}/cards_data")


In [0]:
logger.info("bronze_notebook executed successfully!!")
dbutils.notebook.exit(json.dumps({
    "bronze_path" : f"{bronze_path}/{tax_year}/{client_id}/{date_str}/transactions_data"
}))